<a href="https://colab.research.google.com/github/lisaong/mldds-courseware/blob/master/02_BasicMLAlgorithms/geospatial_feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial
https://www.kaggle.com/camnugent/geospatial-feature-engineering-and-visualization

## Housing prices dataset:
https://www.kaggle.com/camnugent/california-housing-prices

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
data_path = '/content/drive/My Drive/data/california-housing-feature-engineering/'

housing = pd.read_csv(data_path + 'housing.csv')
city_lat_long = pd.read_csv(data_path + 'cal_cities_lat_long.csv')
city_pop_data = pd.read_csv(data_path + 'cal_populations_city.csv')
county_pop_data = pd.read_csv(data_path + 'cal_populations_county.csv')

In [0]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [0]:
city_lat_long.head()

,Name,Latitude,Longitude
0,Adelanto,34.582769,-117.409214
1,Agoura Hills,34.153339,-118.761675
2,Alameda,37.765206,-122.241636
3,Albany,37.886869,-122.297747
4,Alhambra,34.095286,-118.127014


In [0]:
city_lat_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459 entries, 0 to 458
Data columns (total 3 columns):
Name         459 non-null object
Latitude     459 non-null float64
Longitude    459 non-null float64
dtypes: float64(2), object(1)
memory usage: 10.8+ KB


In [0]:
city_pop_data.head()

,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010
0,San Bernardino,Adelanto,1970,2164.0,8517,18130,31765
1,Los Angeles,Agoura Hills,1982,NaN,20390,20537,20330
2,Alameda,Alameda,1854,63852.0,76459,72259,73812
3,Alameda,Albany,1908,15130.0,16327,16444,18539
4,Los Angeles,Alhambra,1903,64767.0,82106,85804,83089


In [0]:
city_pop_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455 entries, 0 to 454
Data columns (total 7 columns):
County                 455 non-null object
City                   455 non-null object
Incorportation_date    455 non-null object
pop_april_1980         425 non-null float64
pop_april_1990         455 non-null int64
pop_april_2000         455 non-null int64
pop_april_2010         455 non-null int64
dtypes: float64(1), int64(3), object(3)
memory usage: 25.0+ KB


Only use cities with populations and coordinates. 

In [0]:
city_coords = city_lat_long.merge(city_pop_data, left_on='Name', right_on='City')
city_coords.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 434 entries, 0 to 433
Data columns (total 10 columns):
Name                   434 non-null object
Latitude               434 non-null float64
Longitude              434 non-null float64
County                 434 non-null object
City                   434 non-null object
Incorportation_date    434 non-null object
pop_april_1980         406 non-null float64
pop_april_1990         434 non-null int64
pop_april_2000         434 non-null int64
pop_april_2010         434 non-null int64
dtypes: float64(3), int64(3), object(4)
memory usage: 37.3+ KB


In [0]:
city_coords.head()

,Name,Latitude,Longitude,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010
0,Adelanto,34.582769,-117.409214,San Bernardino,Adelanto,1970,2164.0,8517,18130,31765
1,Agoura Hills,34.153339,-118.761675,Los Angeles,Agoura Hills,1982,NaN,20390,20537,20330
2,Alameda,37.765206,-122.241636,Alameda,Alameda,1854,63852.0,76459,72259,73812
3,Albany,37.886869,-122.297747,Alameda,Albany,1908,15130.0,16327,16444,18539
4,Alhambra,34.095286,-118.127014,Los Angeles,Alhambra,1903,64767.0,82106,85804,83089


In [0]:
city_coords.drop('City', inplace=True, axis=1)
city_coords.head()

,Name,Latitude,Longitude,County,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010
0,Adelanto,34.582769,-117.409214,San Bernardino,1970,2164.0,8517,18130,31765
1,Agoura Hills,34.153339,-118.761675,Los Angeles,1982,NaN,20390,20537,20330
2,Alameda,37.765206,-122.241636,Alameda,1854,63852.0,76459,72259,73812
3,Albany,37.886869,-122.297747,Alameda,1908,15130.0,16327,16444,18539
4,Alhambra,34.095286,-118.127014,Los Angeles,1903,64767.0,82106,85804,83089


## Distance to nearest town

- Compute distance between two sets of lat, long
- Find out which city is closest to a given coordinate

https://pypi.org/project/geopy/

In [0]:
from geopy.distance import geodesic

In [0]:
city1 = city_coords.iloc[0]['Latitude'], city_coords.iloc[0]['Longitude']
place1 = housing.iloc[0]['latitude'], housing.iloc[0]['longitude']

geodesic(place1, city1).km # distance in km

567.0202547994141

In [0]:
# wrap the above into a function

def compute_distances(row, lat, lon):
  '''Computes distances for a given row from the coordinates dataframe with the
     specified lat and lon
  '''
  return geodesic((row['Latitude'], row['Longitude']), (lat, lon)).km

# apply the function on city_coords and get the nearest city to place1
distances = city_coords.apply(compute_distances, lat=place1[0], lon=place1[1],
                              axis=1)
city_coords.loc[distances.idxmin()]

Name                   Berkeley
Latitude                37.8716
Longitude              -122.273
County                  Alameda
Incorportation_date        1878
pop_april_1980           103328
pop_april_1990           102724
pop_april_2000           102743
pop_april_2010           112580
Name: 32, dtype: object

In [0]:
# find big cities
big_population = 250000

big_city_coords = city_coords[city_coords['pop_april_2010'] > big_population]
big_city_coords

,Name,Latitude,Longitude,County,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010
7,Anaheim,33.835292,-117.914503,Orange,1876,219494.0,266406,328014,336265
22,Bakersfield,35.373292,-119.018711,Kern,1873,105611.0,174820,246889,347483
131,Fresno,36.746842,-119.772586,Fresno,1885,217129.0,354202,427652,494665
209,Long Beach,33.768322,-118.195617,Los Angeles,i. 1888/ d.1896/ \nri. 1897,361355.0,429433,461522,462257
212,Los Angeles,34.052233,-118.243686,Los Angeles,1850,2968579.0,3485398,3694742,3792621
257,Oakland,37.804364,-122.271114,Alameda,1852,339337.0,372242,399484,390724
313,Riverside,33.953350,-117.396156,Riverside,1883,170591.0,226505,255166,303871
320,Sacramento,38.581572,-121.494400,Sacramento,1850,275741.0,369365,407018,466488
326,San Diego,32.715328,-117.157256,San Diego,1850,875538.0,1110549,1223400,1307402
329,San Francisco,37.774931,-122.419417,San Francisco,1850,678974.0,723959,776733,805235


In [0]:
# apply the function on big_city_coords and get the nearest city to place1
distances = big_city_coords.apply(compute_distances, lat=place1[0], lon=place1[1],
                                  axis=1)
big_city_coords.loc[distances.idxmin(), ['Name', 'County', 'pop_april_2010']]

Name              Oakland
County            Alameda
pop_april_2010     390724
Name: 257, dtype: object

In [0]:
# Apply to housing dataframe by converting the above cell into a function

def find_nearest_city(row, coords):
  '''Find the nearest city to the given row, among the list of cities
     and coordinates
  '''
  distances = coords.apply(compute_distances, lat=row['latitude'],
                           lon=row['longitude'], axis=1)
  return coords.loc[distances.idxmin(),  ['Name', 'County', 'pop_april_2010']]

# apply the function on housing (try first 10 rows)
nearest_cities = housing.iloc[:10].apply(find_nearest_city, coords=city_coords, axis=1)
nearest_cities

,Name,County,pop_april_2010
0,Berkeley,Alameda,112580
1,Orinda,Contra Costa,17643
2,Piedmont,Alameda,10667
3,Berkeley,Alameda,112580
4,Berkeley,Alameda,112580
5,Berkeley,Alameda,112580
6,Piedmont,Alameda,10667
7,Piedmont,Alameda,10667
8,Emeryville,Alameda,10080
9,Piedmont,Alameda,10667


In [0]:
# Parallize

import dask.dataframe as dd
from dask.multiprocessing import get

dd_housing = dd.from_pandas(housing, npartitions=10)

In [0]:
# this will take a while to run (around 30 minutes on Colab)
res = dd_housing.map_partitions(lambda df: df.apply(find_nearest_city, coords=city_coords, axis=1)).compute(scheduler='threads')


In [0]:
res.head()

,Name,County,pop_april_2010
0,Berkeley,Alameda,112580
1,Orinda,Contra Costa,17643
2,Piedmont,Alameda,10667
3,Berkeley,Alameda,112580
4,Berkeley,Alameda,112580


In [0]:
res.columns = ['nearest_city_name', 'nearest_city_county', 'nearest_city_pop_april_2010']
res.head()

,nearest_city_name,nearest_city_county,nearest_city_pop_april_2010
0,Berkeley,Alameda,112580
1,Orinda,Contra Costa,17643
2,Piedmont,Alameda,10667
3,Berkeley,Alameda,112580
4,Berkeley,Alameda,112580


In [0]:
housing = pd.concat([housing, res], axis=1)
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,nearest_city_name,nearest_city_county,nearest_city_pop_april_2010
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,Berkeley,Alameda,112580
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,Orinda,Contra Costa,17643
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,Piedmont,Alameda,10667
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,Berkeley,Alameda,112580
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,Berkeley,Alameda,112580


In [0]:
housing.to_csv('/content/drive/My Drive/data/housing_nearest_city.csv', index=None)

In [0]:
res_big = dd_housing.map_partitions(lambda df: df.apply(find_nearest_city, coords=big_city_coords, axis=1)).compute(scheduler='threads')
res_big.head()

,Name,County,pop_april_2010
0,Oakland,Alameda,390724
1,Oakland,Alameda,390724
2,Oakland,Alameda,390724
3,Oakland,Alameda,390724
4,Oakland,Alameda,390724


In [0]:
res_big.columns = ['nearest_big_city_name', 'nearest_big_city_county', 'nearest_big_city_pop_april_2010']
res_big.head()

,nearest_big_city_name,nearest_big_city_county,nearest_big_city_pop_april_2010
0,Oakland,Alameda,390724
1,Oakland,Alameda,390724
2,Oakland,Alameda,390724
3,Oakland,Alameda,390724
4,Oakland,Alameda,390724


In [0]:
housing = pd.concat([housing, res_big], axis=1)
housing.to_csv('/content/drive/My Drive/data/housing_nearest_big_city.csv', index=None)

In [0]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,nearest_city_name,nearest_city_county,nearest_city_pop_april_2010,nearest_big_city_name,nearest_big_city_county,nearest_big_city_pop_april_2010
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,Berkeley,Alameda,112580,Oakland,Alameda,390724
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,Orinda,Contra Costa,17643,Oakland,Alameda,390724
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,Piedmont,Alameda,10667,Oakland,Alameda,390724
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,Berkeley,Alameda,112580,Oakland,Alameda,390724
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,Berkeley,Alameda,112580,Oakland,Alameda,390724
